In [1]:
!pip install skfeature-chappers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.utils import resample
from skfeature.function.similarity_based import fisher_score
import numpy as np
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 2.5 MB/s eta 0:00:00


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import random
import math,time,sys, os
from matplotlib import pyplot
from datetime import datetime

In [3]:
## -----------------Reading dataset files------------------------
train_data=pd.read_csv('/kaggle/input/labels/Combined_inter/Combined_inter/train.csv', index_col=False, header = None)
y_train = train_data[train_data.columns[-1]]
(noTrainRows, _) = train_data.shape

#Converting dataseries into pandas dataframe
y_train = y_train.to_frame()
dataframe_train = train_data.drop(train_data.columns[[-1]], axis=1)

test_data=pd.read_csv('/kaggle/input/labels/Combined_inter/Combined_inter/test.csv', index_col=False, header = None)
y_test = test_data[test_data.columns[-1]]

#Converting dataseries in pandas dataframe
y_test = y_test.to_frame()
dataframe_test = test_data.drop(test_data.columns[[-1]], axis=1)
print('Train and Test datasize : ', dataframe_train.shape, dataframe_test.shape)

Train and Test datasize :  (5876, 4352) (518, 4352)


In [5]:
coln=dataframe_train.columns
coln1=y_train.columns

In [6]:
dataframe=dataframe_train.append(dataframe_test,ignore_index=True)
y=y_train.append(y_test,ignore_index=True)
print('Combined size : ',dataframe.shape)

Combined size :  (6394, 4352)


/tmp/ipykernel_32/3658989962.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe=dataframe_train.append(dataframe_test,ignore_index=True)
/tmp/ipykernel_32/3658989962.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y=y_train.append(y_test,ignore_index=True)


In [7]:
##-----------------Removing columns contains all '0' -------------------
cols=[]
for i in range(len(dataframe.columns)):
    col1=dataframe[dataframe.columns[i]].to_numpy()
    if(np.sum(col1)==0):
        cols.append(dataframe.columns[i])

dataframe = dataframe.drop(columns=cols)
print('Size after removing unnecessary columns :', dataframe.shape)

Size after removing unnecessary columns : (6394, 4063)


In [8]:
df=dataframe.copy()
df['labels']=y
print('After adding labels : ', df.shape)
df = df.replace(np.nan, 0)
df.head()

After adding labels :  (6394, 4064)


,4,5,8,10,11,13,14,17,21,27,...,4343,4344,4345,4346,4347,4348,4349,4350,4351,labels
0,0.011616,0.138488,0.109463,0.0,0.0,0.003859,0.001707,0.115047,0.0,0.000000,...,0.002868,0.000000,0.143688,0.013709,0.103574,0.023923,0.0,0.010077,0.180342,1.0
1,0.013577,0.167723,0.095821,0.0,0.0,0.006735,0.001554,0.136256,0.0,0.000000,...,0.000000,0.000000,0.094773,0.000000,0.062565,0.004175,0.0,0.000000,0.110781,1.0
2,0.010126,0.113474,0.142226,0.0,0.0,0.001838,0.000835,0.093333,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000106,0.000000,0.001493,0.0,0.000000,0.000000,1.0
3,0.009892,0.130496,0.076079,0.0,0.0,0.004165,0.001042,0.105777,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.0
4,0.006942,0.210991,0.133701,0.0,0.0,0.016510,0.001624,0.171192,0.0,0.000004,...,0.008000,0.000859,0.000147,0.026031,0.001614,0.036298,0.0,0.019312,0.000402,1.0


In [9]:
x=df.iloc[:,:-1]
x.head()

,4,5,8,10,11,13,14,17,21,27,...,4342,4343,4344,4345,4346,4347,4348,4349,4350,4351
0,0.011616,0.138488,0.109463,0.0,0.0,0.003859,0.001707,0.115047,0.0,0.000000,...,0.031467,0.002868,0.000000,0.143688,0.013709,0.103574,0.023923,0.0,0.010077,0.180342
1,0.013577,0.167723,0.095821,0.0,0.0,0.006735,0.001554,0.136256,0.0,0.000000,...,0.009817,0.000000,0.000000,0.094773,0.000000,0.062565,0.004175,0.0,0.000000,0.110781
2,0.010126,0.113474,0.142226,0.0,0.0,0.001838,0.000835,0.093333,0.0,0.000000,...,0.004493,0.000000,0.000000,0.000000,0.000106,0.000000,0.001493,0.0,0.000000,0.000000
3,0.009892,0.130496,0.076079,0.0,0.0,0.004165,0.001042,0.105777,0.0,0.000000,...,0.004319,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,0.006942,0.210991,0.133701,0.0,0.0,0.016510,0.001624,0.171192,0.0,0.000004,...,0.051971,0.008000,0.000859,0.000147,0.026031,0.001614,0.036298,0.0,0.019312,0.000402


In [10]:
ln=df.shape[1]
str=df.columns[ln-1]
print(ln, str)

4064 labels


In [11]:
print(x.shape)
dataset=x.to_numpy()

(6394, 4063)


In [12]:
def sort_agents(agents, obj,train_X,val_X,train_Y,val_Y):
    # sort the agents according to fitness
    #train_X, val_X, train_Y, val_Y = data.train_X, data.val_X, data.train_Y, data.val_Y
    (obj_function, weight_acc) = obj

    # if there is only one agent
    if len(agents.shape) == 1:
        num_agents = 1
        fitness = obj_function(agents, train_X, val_X, train_Y, val_Y, weight_acc)
        return agents, fitness

    # for multiple agents
    else:
        num_agents = agents.shape[0]
        fitness = np.zeros(num_agents)
        for id, agent in enumerate(agents):
            fitness[id] = obj_function(agent, train_X, val_X, train_Y, val_Y, weight_acc)
            
        idx = np.argsort(-fitness)
        sorted_agents = agents[idx].copy()
        sorted_fitness = fitness[idx].copy()
        
    return sorted_agents, sorted_fitness

In [13]:
def compute_fitness(agent, train_X, test_X, train_Y, test_Y, weight_acc=0.99):
    # compute a basic fitness measure
    if(weight_acc == None):
        weight_acc = 0.99
    
    weight_feat = 1 - weight_acc
    num_features = len(agent)
   
    acc = compute_accuracy(agent, train_X, test_X, train_Y, test_Y)
    feat = (num_features - np.sum(agent))/num_features

    fitness = weight_acc * acc + weight_feat * feat
    
    return fitness

In [14]:
from sklearn.feature_selection import f_regression

# inputs:
#    X: pandas.DataFrame, features
#    yy: pandas.Series, target variable
#    K: number of features to select


X=x
yy=df[str].to_numpy()
K=x.shape[1]
print(yy[0])
print(X.shape)

# compute F-statistics and initialize correlation matrix
F = pd.Series(f_regression(X, yy)[0], index = X.columns)
corr = pd.DataFrame(.00001, index = X.columns, columns = X.columns)

# initialize list of selected features and list of excluded features
selected = []
not_selected = X.columns.to_list()
Dict={}
# repeat K times
for i in range(K):
  
    # compute (absolute) correlations between the last selected feature and all the (currently) excluded features
    if i > 0:
        last_selected = selected[-1]
        corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
        
    # compute FCQ score for all the (currently) excluded features (this is Formula 2)
    score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
    
    # find best feature, add it to selected and remove it from not_selected
    # print(score[score.argmax()])
    # print(score.max())
    # print(score.index[score.argmax()])
    best = score.index[score.argmax()]
    selected.append(best)
    not_selected.remove(best)
    Dict[best]=score.max()

mrMr=[]
for i in range(x.shape[1]):
  mrMr.append(Dict[x.columns[i]])
#print(mrMr)  

1.0
(6394, 4063)


/tmp/ipykernel_32/3757669169.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
/tmp/ipykernel_32/3757669169.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
/tmp/ipykernel_32/3757669169.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
/tmp/ipykernel_32/3757669169.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when p

In [16]:
for i in range(len(mrMr)):
  mrMr[i]=mrMr[i]/(max(mrMr))

In [17]:
def normalize(arr, t_min, t_max):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return norm_arr

In [18]:
!pip install Py-FS
def fitness_score_values(data,target):
    from Py_FS.filter import PCC as PCC
    from Py_FS.filter import MI as MI
    from sklearn.feature_selection import mutual_info_classif
    from Py_FS.filter import Relief as Relief
    from sklearn.feature_selection import chi2
    
    print(data)
    print(target)
    #print('Pcc scores')
    #PCC_solution = PCC(data, target)
    #print(PCC_solution.scores)
    print('MI scores')
    mi_scores=mutual_info_classif(data, target, discrete_features='auto', n_neighbors=3, copy=True, random_state=None)
    print((mi_scores))
    
    print('Relief scores')
    Relief_solution = Relief(data, target)
    print((Relief_solution.scores))
    
   
    shapley=[]
    b=x.shape[1]
    print(b)
    for i in range(b):
        shapley.append((mi_scores[i]+mrMr[i]+Relief_solution.scores[i])/3)
    return shapley

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 47.5 MB/s eta 0:00:0000:0100:01


In [19]:
target=df[str].to_numpy()
print(len(target), target)

6394 [1. 1. 1. ... 0. 0. 0.]


In [28]:
!pip install tabulate
!pip install ReliefF
score=fitness_score_values(x.values,target)

[[0.01161642 0.13848796 0.10946296 ... 0.         0.01007661 0.18034205]
 [0.01357734 0.16772316 0.09582078 ... 0.         0.         0.11078054]
 [0.01012561 0.11347369 0.14222626 ... 0.         0.         0.        ]
 ...
 [0.0210852  0.07289205 0.14390372 ... 0.         0.         0.        ]
 [0.04416167 0.21579507 0.18120509 ... 0.         0.         0.        ]
 [0.03935141 0.19237143 0.24310282 ... 0.         0.         0.        ]]
[1. 1. 1. ... 0. 0. 0.]
MI scores
[0.03848786 0.04592252 0.00396162 ... 0.01998512 0.04519501 0.0739572 ]
Relief scores
[1.37194581e-01 1.05898361e-02 1.18322191e-04 ... 8.58604981e-01
 7.23244394e-01 6.97745962e-01]
4063


In [29]:
score = np.asarray(score)
print(score.shape)

(4063,)


In [30]:
# Returns the rank array
def ranking(score):
    rank = np.copy(score)
    rank_x = np.sort(score)
    for i in range(len(rank_x)):
        for j in range(len(rank_x)):
            if rank_x[i] == score[j]:
                rank[j] = i
    return np.asarray(rank) #rank

#Takes the ranking array consisting of rank in the positions and creates the 0th iteration feature map
def random_feature(rank, org_feat):
    feat_0 = np.copy(org_feat)
    cutoff = 0.5*(len(rank))
    for i in range(len(rank)):
        if rank[i] < cutoff:
            feat_0[i] = org_feat[i]
        else:
            feat_0[i] = 0
    return feat_0 #0th iteration feature map

#Mutates by deleting selected feature and replacing by non selected ones
def randomize(rank,feat_0,org_feat):
    feat_1 = np.copy(feat_0)
    replace = 0.1*(len(rank)) #count of number of mutation
    for i in range(int(replace)): #masking random selected top feats 
        index = np.random.randint(0.5*(len(rank))) #random index for mutation
        for k in range(len(rank)):
            if rank[k] == index:
                feat_1[index] = 0
    for j in range(len(rank)-int(replace),len(rank)): #top 10% unselected elements are selected
        feat_1[j] = org_feat[j]
    return feat_1 #2nd,3rd,.... iteration feature map

def mask(feat_1):
    mask = np.copy(feat_1)
    for i in range(len(feat_1)):
        if feat_1[i]>0:
            mask[i]=1
        else:
            mask[i]=0
    return mask

In [31]:
omega=0.9
def compute_accuracy(solution, trainX, trainy, testX,testy):
    cols=np.flatnonzero(solution)
    val=1
    if np.shape(cols)[0]==0:
        return val
    clf=KNeighborsClassifier(n_neighbors=7)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    error=1-clf.score(test_data,testy)

    #in case of multi objective  []
    featureRatio = (solution.sum()/np.shape(solution)[0])
    val=omega*error+(1-omega)*featureRatio
    # print(error,featureRatio,val)
    return val 

In [32]:
(a,b)=np.shape(df)
trainX = df.values[:noTrainRows,0:b-1]
testX  = df.values[noTrainRows:,0:b-1]
trainy = target[0:noTrainRows]
testy  = target[noTrainRows:]

In [33]:
#Applying mask on the entire test set and finding accuracy
masked=mask(best)
#TESTX=masked_dataset(masked,dataset)
print(1-(compute_accuracy(best,trainX,trainy,testX,testy)))

0.5965748912918145


In [34]:
rank=ranking(score)
feat_prev=random_feature(rank,x.values[0])
feat_next=randomize(rank,feat_prev,x.values[0])
best=feat_prev
n=1000 #no. of iterations
for i in range (1,n+1):
    print("Iteration:")
    print(i)
    feat=compute_accuracy(feat_prev,trainX,trainy,testX,testy)
    feat1=compute_accuracy(feat_next,trainX,trainy,testX,testy)
    if feat1<feat:
        best=feat_next
        feat_prev=feat_next
    print("Accuracy:")
    print(1-(compute_accuracy(best,trainX,trainy,testX,testy)))
    
    feat_next=randomize(rank,feat_prev,x.values[0])

Iteration:
1
Accuracy:
0.32147033310030004
Iteration:
2
Accuracy:
0.32147033310030004
Iteration:
3
Accuracy:
0.32147033310030004
Iteration:
4
Accuracy:
0.32147033310030004
Iteration:
5
Accuracy:
0.32147033310030004
Iteration:
6
Accuracy:
0.3231531737123021
Iteration:
7
Accuracy:
0.3231531737123021
Iteration:
8
Accuracy:
0.3351151811061802
Iteration:
9
Accuracy:
0.33668903027557584
Iteration:
10
Accuracy:
0.33668903027557584
Iteration:
11
Accuracy:
0.39038862361049065
Iteration:
12
Accuracy:
0.3989705120441761
Iteration:
13
Accuracy:
0.3989705120441761
Iteration:
14
Accuracy:
0.4284163817359503
Iteration:
15
Accuracy:
0.4284163817359503
Iteration:
16
Accuracy:
0.49088505975687524
Iteration:
17
Accuracy:
0.49088505975687524
Iteration:
18
Accuracy:
0.49088505975687524
Iteration:
19
Accuracy:
0.49088505975687524
Iteration:
20
Accuracy:
0.49088505975687524
Iteration:
21
Accuracy:
0.49088505975687524
Iteration:
22
Accuracy:
0.49088505975687524
Iteration:
23
Accuracy:
0.5481364207507313
Itera

In [35]:
#For 100%-4005 top features
print(1-(compute_accuracy(x.values[0],trainX,trainy,testX,testy)))

0.3134169004750428


In [38]:
masked = mask(best)
print(masked)
count=0
for i in range(len(masked)):
    if masked[i]>0:
        count=count+1
print(count)

[0. 0. 0. ... 0. 1. 1.]
420
